[Open with Colab](https://colab.research.google.com/github/1never/UEC_AIX_seminar2020/blob/master/bert_dialogue.ipynb)

In [ ]:
# ライブラリのインストール
!pip install transformers
!git clone https://github.com/huggingface/transformers.git
!apt install git make curl xz-utils file
!apt install mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8
!pip install mecab-python3==0.996.5

In [ ]:
# データのダウンロード
!git clone https://github.com/1never/UEC_AIX_seminar2020.git data

# SF作家 海野十三の小説から抽出した対話データを解凍
!unzip data/unno_pair.zip 

# 青空文庫全データの対話データを解凍
!7za x data/aozora_pair.7z

# 学習済みモデル保存用フォルダの作成
!mkdir bert_data

In [3]:
import random

write_lines = []
uttrs = []

filename = 'unno_pair.txt'
# filename = 'aozora_pair.txt'

with open(filename) as f:
    for l in f:
        l = l.strip()
        if "\t" in l:
            # 実際の応答ペアを正解とし，ラベルは1とする．
            write_lines.append(l + "\t1\n")
            # 不正解ペアの作成のため，発話を保存
            uttrs.append(l.split("\t")[0])
            uttrs.append(l.split("\t")[1])
  
# 正解ペアと同じ数だけ不正解ペアを作成
for i in range(len(write_lines)):
    # ランダムな応答ペアを不正解とし，ラベルは0とする．
    write_lines.append(random.choice(uttrs) + "\t" + random.choice(uttrs) + "\t0\n")
  
 # 正解ペアと不正解ペアが入ったリストをシャッフルする
random.shuffle(write_lines)
  
index = 0
with open("bert_data/dev.tsv", "w") as var_f:
    # 開発データとしてdev.tsvに200行を書き込む．
    for l in write_lines[:200]:
        var_f.write(str(index) + "\t" + l)
        index += 1
index = 0
with open("bert_data/train.tsv", "w") as var_f:
    # 学習データとしてtrain.tsvにのこりを書き込む．
    for l in write_lines[200:]:
        var_f.write(str(index) + "\t" + l)
        index += 1

In [ ]:
# max_stepsの値を大きな値に設定することで，より多くのデータで学習できるが，より多くの時間が必要となる
!python transformers/examples/text-classification/run_glue.py --data_dir bert_data/  --overwrite_output_dir \
--model_name_or_path cl-tohoku/bert-base-japanese-whole-word-masking --task_name WNLI --evaluate_during_training --save_steps 1000 --max_steps 1000 \
--output_dir bert_output/ --do_train --do_eval --per_gpu_train_batch_size 16

In [ ]:
# Elasticsearchのダウンロードと解凍
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz

# Elasticsearchの日本語形態素解析用プラグイン analysis-kuromojiのインストール
!elasticsearch-7.0.0/bin/elasticsearch-plugin install analysis-kuromoji

# Pythonのelasticsearchライブラリのインストール
!pip install elasticsearch

In [6]:
# Elasticsearchの実行
!pkill -f elasticsearch
!chown -R daemon:daemon elasticsearch-7.0.0/bin/
!chown -R daemon:daemon elasticsearch-7.0.0/

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1))

In [ ]:
# 接続テスト (上記セルの実行から30秒ほど待つ必要があります)
!curl -X GET "localhost:9200/"

# Pythonライブラリによる接続テスト
from elasticsearch import Elasticsearch, helpers
es = Elasticsearch()
es.ping()

In [ ]:
# 対話データをElasticsearchにインサート
def load():
    try:
        es.delete_by_query(index='dialogue_pair', body={"query": {"match_all": {}}})
        print("既存データを削除")
    except:
        print("削除対象データなし")
        pass

    with open(filename) as f:
        for i, __ in enumerate(f):
            print(i, '...', end='\r')
            __ = __.split('\t')
            query = __[0].strip()
            response = __[1].strip()
            item = {'_index':'dialogue_pair', '_type':'docs', '_source':{ 'query':query, 'response':response }}
            yield item

print(helpers.bulk(es, load()))

In [ ]:
from transformers.modeling_bert import BertForSequenceClassification
from transformers.tokenization_bert import BertTokenizer
import torch
import torch.nn.functional as F

# 表示する選択肢の数
OPTION_NUM = 10

# Elasticsearchで検索する数(多くすると計算に時間がかかるようになります)
SEARCH_NUM = 50

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class BertEvaluator:
    def __init__(self):
        # 事前学習済みのトークナイザとモデルをロード
        self.tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', do_lower_case=False)
        self.model = BertForSequenceClassification.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', num_labels=2)
        
        # Google Colabでファインチューニングしたモデルをロード
        self.model.load_state_dict(torch.load("bert_output/pytorch_model.bin", map_location="cpu"))
        self.model.to(device)

    def evaluate(self, user_input, candidate):
        with torch.no_grad():
            # 発話のペアを特徴ベクトルに変換
            tokenized = self.tokenizer([[user_input, candidate]], return_tensors="pt")
            input_ids = tokenized["input_ids"].to(device)
            token_type_ids = tokenized["token_type_ids"].to(device)

            # ファインチューニング済みのBERTを用いて特徴ベクトルから2文のスコアを計算
            result = self.model.forward(input_ids, token_type_ids=token_type_ids)
            # softmax関数によりスコアを正規化
            result = F.softmax(result[0], dim=1).cpu().numpy().tolist()

            # 結果を返す．
            return result[0][1]

# プログラムがエラーで落ちた場合，一時的にElasticsearchに接続できなくなりますが，一定時間経つことで接続可能になります．
es = Elasticsearch()
be = BertEvaluator()
def get_reply(utterance, size=SEARCH_NUM):
    results = es.search(index='dialogue_pair', body={'query':{'match':{'query':utterance}}, 'size':size,})

    tmp_dict = {}
    for r in results['hits']['hits']:
        score = be.evaluate(utterance, r['_source']['response'])
        tmp_dict[r['_source']['response']] = score
    score_sorted = sorted(tmp_dict.items(), key=lambda x:x[1]*-1.0)
    return [x[0] for x in score_sorted]

res = None
logs = []
while(True):
    u = input("\n>")
    if "exit" == u:
        break
    elif u.isdecimal() and res is not None and int(u) < len(res):
        u = res[int(u)]
    elif "back" == u:
        if len(logs) > 1:
            logs.pop()
            u = logs.pop()
        else:
            logs.pop()
            continue


    res = get_reply(u)
    logs.append(u)
    for i, l in enumerate(logs):
        print("log " + str(i) + ": " , l)
    for i, r in enumerate(res):
        print(i, r)
        if i >= OPTION_NUM:
          break

# 使用方法
# 1. ">"の右の入力欄にセリフを入力します
# 2. 現在までのログと検索された候補が表示されます．
# 3. 表示された候補の左の数字を">"の右の入力欄に入力することでその候補が次のセリフになります．数字以外のものを入力するとそれがセリフになります．
# 4. 入力を間違えた場合，「back」と入力すると前回の状態に戻ることができます．
# 5. 「exit」と入力すると終了します．